In [18]:
import polars as pl
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from collections import defaultdict
import json
from pathlib import Path
from typing import Dict, Any, Optional
import plotly.io

plotly.io.templates.default = "seaborn"

OUT_DIR = Path("..", "data", "data_analysis")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [19]:
data = pl.read_parquet("../data/products.parquet")

In [20]:
data.head(5)

url,main_image_url,other_image_urls,category,gender,rating,rating_count,title,description
str,str,list[str],str,str,f64,i64,str,str
"""https://www.in…","""https://www.in…","[""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw33243a42/images/iic-asics-1011b430-401-detail-x-0002.jpg?sw=580&sh=580&sm=fit"", ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dwfb79fa40/images/iic-asics-1011b430-401-back-x-0001.jpg?sw=580&sh=580&sm=fit"", … ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw97d3eddd/images/iic-asics-1011b430-401-left-x-0001.jpg?sw=580&sh=580&sm=fit""]","""hardloopschoen…","""heren""",4.5,2,""" Asics · Gel-…",""" Product…"
"""https://www.in…","""https://www.in…","[""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw5a02c429/images/iic-brooks-1103931d-438-dynamic-ies-0001.jpg?sw=580&sh=580&sm=fit"", ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dwe6c5e2d3/images/iic-brooks-1103931d-438-left-ies-0001.jpg?sw=580&sh=580&sm=fit"", … ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw92995bae/images/iic-brooks-1103931d-438-top-ies-0001.jpg?sw=580&sh=580&sm=fit""]","""hardloopschoen…","""heren""",4.4,696,""" Brooks · Gho…",""" Product…"
"""https://www.in…","""https://www.in…","[""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw066984d5/images/iic-asics-1011b621-400-backpair-x-0001.jpg?sw=580&sh=580&sm=fit"", ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dwa8c6296f/images/iic-asics-1011b621-400-dynamicpair-x-0002.jpg?sw=580&sh=580&sm=fit"", … ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dwde300fb4/images/iic-asics-1011b621-400-sole-x-0001.jpg?sw=580&sh=580&sm=fit""]","""hardloopschoen…","""heren""",4.7,55,""" Asics · Gel-…",""" Product…"
"""https://www.in…","""https://www.in…","[""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dwd1b599eb/images/iic-asics-1011b621-002-dynamicpair-x-0001.jpg?sw=580&sh=580&sm=fit"", ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw588eda07/images/iic-asics-1011b621-002-solepair-x-0001.jpg?sw=580&sh=580&sm=fit"", … ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw150ef3be/images/iic-asics-1011b621-002-backpair-x-0001.jpg?sw=580&sh=580&sm=fit""]","""hardloopschoen…","""heren""",4.7,55,""" Asics · Gel-…",""" Product…"
"""https://www.in…","""https://www.in…","[""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw1ed8e720/images/iic-brooks-1103911d-006-detail-x-0004.jpg?sw=580&sh=580&sm=fit"", ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw8072470a/images/iic-brooks-1103911d-006-detail-x-0001.jpg?sw=580&sh=580&sm=fit"", … ""https://www.intersport.nl/dw/image/v2/BBWT_PRD/on/demandware.static/-/Sites-intersport-master-catalog/default/dw5d1e2048/images/iic-brooks-1103911d-006-left-x-0001.jpg?sw=580&sh=580&sm=fit""]","""hardloopschoen…","""heren""",null,0,""" Brooks · Adr…",""" Product…"


In [21]:
# See how many values are missing
data.null_count()

url,main_image_url,other_image_urls,category,gender,rating,rating_count,title,description
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,721,0,0,132


In [22]:
# Now let's inspect unique values
data.select(pl.exclude("other_image_urls").n_unique())

url,main_image_url,category,gender,rating,rating_count,title,description
u32,u32,u32,u32,u32,u32,u32,u32
1429,1429,7,2,24,202,871,794


In [40]:
import re

result = []
for s in data["description"]:
    if s is None:
        continue
    if not re.match(r"^\s*$", s):
        result.append(s)

len(result)

1435

In [43]:
def value_counts(df: pl.DataFrame, col: str) -> Dict[Any, int]:
    result = defaultdict(int)
    for item in df[col]:
        result[item] += 1
    return dict(result)


# Interestingly enough there are some items with duplicate titles and descriptions. Those are inspected (in external file)
title_val_counts = value_counts(data, "title")
description_val_counts = value_counts(data, "description")

# Select only the intereseting columns
_interesting_cols = ["url", "category", "gender", "title", "description"]

duplicate_title_data = (
    data.filter(pl.col("title").map_dict(title_val_counts, return_dtype=pl.Int64) > 1)
    .select(_interesting_cols)
    .sort("title")
)
duplicate_description_data = (
    data.filter(
        pl.col("description").map_dict(description_val_counts, return_dtype=pl.Int64)
        > 1
    )
    .select(_interesting_cols)
    .sort("description")
)

# Write to excel for easy reading
duplicate_title_data.write_csv(OUT_DIR / "duplicate_title_data.csv")
duplicate_description_data.write_csv(OUT_DIR / "duplicate_description_data.csv")

# The items with duplicate titles and descriptions are often the same item with different colors

In [24]:
def plot_value_counts(
    df: pl.DataFrame, col: str, title: Optional[str] = None
) -> go.Figure:
    vcounts = value_counts(df, col)

    plot_dict = [{"value": v, "count": c} for v, c in vcounts.items()]

    plot_df = pl.DataFrame(plot_dict).to_pandas()

    fig = px.bar(plot_df, x="value", y="count", title=title)

    return fig


cat_plot = plot_value_counts(data, "category", title="Categorie verdeling")
gender_plot = plot_value_counts(data, "gender", title="Geslacht verdeling")

cat_plot.show()
gender_plot.show()

# There are very little cycling shoes
# There are more shoes for males than females